In [1]:
# TO DO: confirm validation is "group" and set up training loop on matches 2011-2013, test 2014

In [2]:
# determine the optimal pre-match forecast, given elo_diff and s_elo_diff
# step 3: calculate glicko and s_glicko for each player and do the same

In [68]:
import sys
sys.path.insert(0,'/Users/jacobgollub/Desktop/college (current)/research/pbp_explorations/scripts/sackmann')
import tennisGameProbability,tennisMatchProbability,tennisSetProbability,tennisTiebreakProbability
from tennisMatchProbability import matchProb
from helper_functions import validate_results
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss,accuracy_score

# can test this on our subset of 10,000 matches as well as all matches in the database:
df = pd.read_csv('../my_data/elo_atp_matches_all_10_1.csv')
del df['Unnamed: 0']

# currently looking at 2014 tour-level matches, excluding Davis Cup
#df = df[df['match_year']==2014].reset_index(drop=True)
#df = df[df['match_year'].isin([2013,2014,2015,2016])]
df = df[df['tny_name']!='Davis Cup'].reset_index(drop=True)
df.head(2)

,tny_id,tny_name,surface,tny_date,match_year,match_month,p0_name,p1_name,p0_elo,p1_elo,...,match_prob_kls,match_prob_kls_JS,match_prob_sf_kls,match_prob_sf_kls_JS,match_prob_adj_kls,match_prob_adj_kls_JS,elo_prob,elo_prob_538,sf_elo_prob,sf_elo_prob_538
0,2000-339,Adelaide,Hard,2000-01-03,2000,1,Arnaud Clement,Thomas Enqvist,1675.243826,1921.802173,...,0.248213,0.255890,0.349917,0.354126,0.223227,0.224541,0.194771,0.204867,0.211304,0.233946
1,2000-339,Adelaide,Hard,2000-01-03,2000,1,Roger Federer,Jens Knippschild,1664.533683,1697.781488,...,0.346116,0.358355,0.479730,0.476991,0.431598,0.432470,0.452298,0.518498,0.546058,0.594731


In [73]:
sub = df[df['match_year']==2010]

In [74]:
np.mean(sub['p0_52_s_adj'])

0.048064498788131446

In [86]:
s1 = df[u'p0_s_kls_adj_JS']+df[u'p1_s_kls_adj_JS']
s2 = df[u'p0_s_kls']+df[u'p1_s_kls']

np.array([s1,s2]).T[5:]

array([[ 1.32535931,  1.30092997],
       [ 1.19611161,  1.21545739],
       [ 1.21602722,  1.19830541],
       ..., 
       [ 1.38458587,  1.38173733],
       [ 1.24064612,  1.24840876],
       [ 1.29853796,  1.30590966]])

In [78]:
df[['p0_name','p1_name',u'p0_s_kls_adj_JS', u'p1_s_kls_adj_JS',u'p0_s_kls', u'p1_s_kls']]

,p0_name,p1_name,p0_s_kls_adj_JS,p1_s_kls_adj_JS,p0_s_kls,p1_s_kls
0,Arnaud Clement,Thomas Enqvist,0.584848,0.644136,0.592318,0.645791
1,Roger Federer,Jens Knippschild,0.645108,0.658910,0.627812,0.659661
2,Jan Michael,Wayne Arthurs,0.742533,0.744822,0.747076,0.762827
3,Sebastien Grosjean,Andrew Ilie,0.677996,0.597805,0.654018,0.608087
4,Magnus Norman,Scott Draper,0.701908,0.590746,0.692391,0.599312
5,Richard Fromberg,Todd Woodbridge,0.694407,0.630952,0.659751,0.641179
6,Gianluca Pozzi,James Sekulov,0.576428,0.619683,0.599352,0.616105
7,Alberto Martin,Vincent Spadea,0.588425,0.627602,0.585965,0.612341
8,Lleyton Hewitt,Mark Woodforde,0.674087,0.588330,0.671552,0.588731
9,Stephane Huet,Dejan Petrovic,0.634751,0.600492,0.628828,0.654364


In [62]:
from scipy.stats import pearsonr
x = pearsonr(df['match_prob_adj_kls_JS'],df['elo_prob'])[0]
x**2

0.75566289673683629

In [57]:
sub = df[df['match_year']==2014]
sub = df[['match_prob_kls_JS','match_prob_adj_kls_JS','elo_prob','winner']]
sub

,match_prob_kls_JS,match_prob_adj_kls_JS,elo_prob,winner
0,0.416493,0.438381,0.264745,1
1,0.568344,0.575419,0.592726,0
2,0.518029,0.477793,0.426908,1
3,0.351763,0.191347,0.215464,1
4,0.551346,0.572378,0.564399,1
5,0.618387,0.441432,0.207918,1
6,0.198096,0.142232,0.169576,0
7,0.386719,0.317827,0.469735,1
8,0.227187,0.587067,0.564702,1
9,0.345756,0.332295,0.291583,0


In [38]:
df.shape

(2409, 92)

In [39]:
from scipy.stats import norm
for col in ['match_prob_kls','match_prob_kls_JS','match_prob_adj_kls_JS']:
    df[col.replace('prob','z')] = norm.ppf(df[col])
print df.shape

(2409, 95)


In [40]:
print 'elo baseline: ',  sum((df['elo_diff']>0) == df['winner'])/float(len(df))
print log_loss(df['winner'],[(1+10**(diff/-400.))**-1 for diff in df['elo_diff']])
print log_loss(df['winner'],[(1+10**(diff/-400.))**-1 for diff in df['sf_elo_diff']])
print 'surface elo baseline: ', sum((df['sf_elo_diff']>0) == df['winner'])/float(len(df))
print 'elo 538 baseline: ',  sum((df['elo_diff_538']>0) == df['winner'])/float(len(df))
print log_loss(df['winner'],[(1+10**(diff/-400.))**-1 for diff in df['elo_diff_538']])
print log_loss(df['winner'],[(1+10**(diff/-400.))**-1 for diff in df['sf_elo_diff_538']])
print 'surface elo 538 baseline: ', sum((df['sf_elo_diff_538']>0) == df['winner'])/float(len(df))

elo baseline:  0.690743046907
0.585792277895
0.590953331806
surface elo baseline:  0.684101286841
elo 538 baseline:  0.692403486924
0.586512005422
0.592019120299
surface elo 538 baseline:  0.694063926941


In [41]:
current = (0,0)
for weight in np.arange(11)*.1:
    accuracy = sum(((1-weight)*df['elo_diff_538']+weight*df['sf_elo_diff_538']>0) == df['winner'])/float(len(df))
    if accuracy > current[1]:
        current=weight,accuracy
print 'best s_elo weight: ',current[0],'accuracy =',current[1]

best s_elo weight:  0.9 accuracy = 0.69904524699


In [ ]:
df[[]]

In [53]:
sub_df = df[(df['match_prob_adj_kls_JS']>.1) & (df['match_prob_adj_kls_JS']<.9)]
sub_df = sub_df.reset_index(drop=True)

In [ ]:
df['best_prob'] = np.zeros(len(df))
elo_ind = df[df[]]
df['best_prob'].loc[elo_ind] = df['elo_prob'].loc[elo_ind]

In [56]:
cols = [['elo_diff'],['sf_elo_diff'],['elo_diff_538'],['sf_elo_diff_538'],
        ['elo_diff','sf_elo_diff'],['elo_diff_538','sf_elo_diff_538'],\
        ['elo_diff','sf_elo_diff','match_z_kls'],\
        ['elo_diff_538','sf_elo_diff_538','match_z_kls']]
probs = [u'match_prob_kls',u'match_prob_kls_JS', u'match_prob_sf_kls', u'match_prob_sf_kls_JS',
        'match_prob_adj_kls','match_prob_adj_kls_JS']
probs = [u'match_prob_kls',u'match_prob_kls_JS', u'match_prob_sf_kls', u'match_prob_sf_kls_JS',
        'match_prob_adj_kls','match_prob_adj_kls_JS']
n_splits = 5
validate_results(df,probs=probs,lm_columns=cols,n_splits=n_splits)

match_prob_kls
accuracy:  0.647572053381
loss:  0.649046110638
match_prob_kls_JS
accuracy:  0.654217958782
loss:  0.615574735215
match_prob_sf_kls
accuracy:  0.633458993625
loss:  0.70745443256
match_prob_sf_kls_JS
accuracy:  0.635948620181
loss:  0.638947479306
match_prob_adj_kls
accuracy:  0.678286074137
loss:  0.631652725284
match_prob_adj_kls_JS
accuracy:  0.678701011896
loss:  0.617003440797
lm columns:  ['elo_diff']
accuracy:  0.691982470821
loss:  0.583494023867
lm columns:  ['sf_elo_diff']
accuracy:  0.682849526833
loss:  0.590019820668
lm columns:  ['elo_diff_538']
accuracy:  0.689490256295
loss:  0.58131848142
lm columns:  ['sf_elo_diff_538']
accuracy:  0.692401721862
loss:  0.58574530602
lm columns:  ['elo_diff', 'sf_elo_diff']
accuracy:  0.691569258374
loss:  0.580312527405
lm columns:  ['elo_diff_538', 'sf_elo_diff_538']
accuracy:  0.693232460037
loss:  0.577122858823
lm columns:  ['elo_diff', 'sf_elo_diff', 'match_z_kls']
accuracy:  0.69032358244
loss:  0.580122364088
lm 

In [44]:
cols = [['elo_diff'],['sf_elo_diff'],['elo_diff_538'],['sf_elo_diff_538'],
        ['elo_diff','sf_elo_diff'],['elo_diff_538','sf_elo_diff_538'],\
        ['elo_diff','sf_elo_diff','match_z_kls'],\
        ['elo_diff_538','sf_elo_diff_538','match_z_kls']]
probs = [u'match_prob_kls',u'match_prob_kls_JS', u'match_prob_sf_kls', u'match_prob_sf_kls_JS',
        'match_prob_adj_kls','match_prob_adj_kls_JS']
probs = [u'match_prob_kls',u'match_prob_kls_JS', u'match_prob_sf_kls', u'match_prob_sf_kls_JS',
        'match_prob_adj_kls','match_prob_adj_kls_JS']
n_splits = 5
validate_results(sub_df,probs=probs,lm_columns=cols,n_splits=n_splits)

match_prob_kls
accuracy:  0.64756256416
loss:  0.649041394428
match_prob_kls_JS
accuracy:  0.654205018935
loss:  0.615580441537
match_prob_sf_kls
accuracy:  0.633465894877
loss:  0.707457342054
match_prob_sf_kls_JS
accuracy:  0.635955521433
loss:  0.638949580237
match_prob_adj_kls
accuracy:  0.678284348824
loss:  0.63164397796
match_prob_adj_kls_JS
accuracy:  0.67870014924
loss:  0.617004525426
lm columns:  ['elo_diff']
accuracy:  0.691568395718
loss:  0.584374027562
lm columns:  ['sf_elo_diff']
accuracy:  0.683681990321
loss:  0.590696836947
lm columns:  ['elo_diff_538']
accuracy:  0.689078769162
loss:  0.582284147323
lm columns:  ['sf_elo_diff_538']
accuracy:  0.690743696138
loss:  0.586415368616
lm columns:  ['elo_diff', 'sf_elo_diff']
accuracy:  0.690734206917
loss:  0.581186728955
lm columns:  ['elo_diff_538', 'sf_elo_diff_538']
accuracy:  0.693642221858
loss:  0.578150669097
lm columns:  ['elo_diff', 'sf_elo_diff', 'match_z_kls']
accuracy:  0.689487668326
loss:  0.581888496203
lm

In [12]:
#df.to_csv('../my_data/elo_atp_matches_21st_century_9_12.csv')

In [21]:
# try out different weightings of normalized surface serve stats...
# probs = []
# for p in [.1,.2,.3,.4,.5,.6,.7,.8,.9]:
#     wgt_s0 = (1-p)*df['p0_s_pct_JS'] + p*df['p0_sf_s_pct_JS']
#     wgt_s1 = (1-p)*df['p1_s_pct_JS'] + p*df['p1_sf_s_pct_JS']
#     wgt_r0 = (1-p)*df['p0_r_pct_JS'] + p*df['p0_sf_r_pct_JS']
#     wgt_r1 = (1-p)*df['p1_r_pct_JS'] + p*df['p1_sf_r_pct_JS']
#     df['p0_s_wgt'+str(p)] = df['tny_stats']+(wgt_s0-df['avg_52_s']) - (wgt_r1-df['avg_52_r'])
#     df['p1_s_wgt'+str(p)] = df['tny_stats']+(wgt_s1-df['avg_52_s']) - (wgt_r0-df['avg_52_r'])
    
#     df['match_prob_wgt_'+str(p)] = [matchProb(row['p0_s_wgt'+str(p)],1-row['p1_s_wgt'+str(p)]) \
#                                     for i,row in df.iterrows()]
#     probs.append('match_prob_wgt_'+str(p))

In [161]:
# one-time fix to transfer over the elo_diff's
# import pandas as pd
# import numpy as np

# df_og = pd.read_csv('../my_data/elo_pbp_with_surface_8_22.csv')
# del df_og['Unnamed: 0']
# df = pd.read_csv('../my_data/feature_df_pbp3_8_23_alphas.csv')
# del df['Unnamed: 0']

# elo_dict = dict(zip(range(len(df_og)),df_og['elo_diff']))
# s_elo_dict = dict(zip(range(len(df_og)),df_og['s_elo_diff']))

# df['elo_diff'] = [elo_dict[m_id] for m_id in df['match_id']]
# df['s_elo_diff'] = [s_elo_dict[m_id] for m_id in df['match_id']]
# df.to_csv('../my_data/feature_df_pbp3_8_23_alphas.csv')

In [ ]:
# from sklearn import linear_model
# from sklearn.model_selection import KFold
# from sklearn.metrics import log_loss,accuracy_score

# n_splits = 5
# kfold = KFold(n_splits=n_splits,shuffle=True)
# scores = np.zeros([3,2,n_splits]);i=0
# cols = ['elo_diff','s_elo_diff']
# for train_ind,test_ind in kfold.split(df):
#     lm = linear_model.LogisticRegression(fit_intercept = True)
#     train_df,test_df = df.loc[train_ind],df.loc[test_ind]
#     lm.fit(train_df[cols].values.reshape([len(train_df),len(cols)]),train_df['winner'])
#     y_preds = lm.predict(test_df[cols].values.reshape([len(test_df),len(cols)]))
#     y_preds2 = test_df['match_prob_kls']>.5
#     y_preds3 = test_df['match_prob_kls_JS']>.5
#     y_probs = lm.predict_proba(test_df[cols].values.reshape([len(test_df),len(cols)]))
#     scores[0][0][i]=accuracy_score(test_df['winner'],y_preds)
#     scores[0][1][i]=log_loss(test_df['winner'],y_probs,labels=[0,1])
#     scores[1][0][i]=accuracy_score(test_df['winner'],y_preds2)
#     scores[1][1][i]=log_loss(test_df['winner'],test_df['match_prob_kls'],labels=[0,1])
#     scores[2][0][i]=accuracy_score(test_df['winner'],y_preds3)
#     scores[2][1][i]=log_loss(test_df['winner'],test_df['match_prob_kls_JS'],labels=[0,1])

    
#     i+=1
# print '% s_elo used in lm fit: ',lm.coef_[0][1]/(lm.coef_[0][0]+lm.coef_[0][1])
# print 'accuracy: ', np.mean(scores[0][0])
# print 'loss: ', np.mean(scores[0][1])

# print 'kls probabilities'
# print 'accuracy: ', np.mean(scores[1][0])
# print 'loss: ', np.mean(scores[1][1])

# print 'kls JS probabilities'
# print 'accuracy: ', np.mean(scores[2][0])
# print 'loss: ', np.mean(scores[2][1])

In [ ]:
# from sklearn import linear_model
# from sklearn.model_selection import KFold
# from sklearn.metrics import log_loss,accuracy_score

# # cols is a list of all column sets to test; compare with kls pre-match forecasts
# def validate_results(df,columns,n_splits=5):
#     kfold = KFold(n_splits=n_splits,shuffle=True)
#     scores = np.zeros([len(columns)+2,2,n_splits]);i=0
#     for train_ind,test_ind in kfold.split(df):
#         lm = linear_model.LogisticRegression(fit_intercept = True)
#         train_df,test_df = df.loc[train_ind],df.loc[test_ind]
        
#         for k,cols in enumerate(columns):
#             lm.fit(train_df[cols].values.reshape([len(train_df),len(cols)]),train_df['winner'])
#             y_preds = lm.predict(test_df[cols].values.reshape([len(test_df),len(cols)]))
#             y_probs = lm.predict_proba(test_df[cols].values.reshape([len(test_df),len(cols)]))
#             scores[k][0][i]=accuracy_score(test_df['winner'],y_preds)
#             scores[k][1][i]=log_loss(test_df['winner'],y_probs,labels=[0,1])
        
#         y_preds2 = test_df['match_prob_kls']>.5
#         y_preds3 = test_df['match_prob_kls_JS']>.5
#         scores[len(columns)][0][i]=accuracy_score(test_df['winner'],y_preds2)
#         scores[len(columns)][1][i]=log_loss(test_df['winner'],test_df['match_prob_kls'],labels=[0,1])
#         scores[len(columns)+1][0][i]=accuracy_score(test_df['winner'],y_preds3)
#         scores[len(columns)+1][1][i]=log_loss(test_df['winner'],test_df['match_prob_kls_JS'],labels=[0,1])
#         i+=1
    
#     for i,cols in enumerate(columns):
#         print 'columns: ',cols
#         #print '% s_elo used in lm fit: ',lm.coef_[0][1]/(lm.coef_[0][0]+lm.coef_[0][1])
#         print 'accuracy: ', np.mean(scores[i][0])
#         print 'loss: ', np.mean(scores[i][1])
    
#     print 'kls probabilities'
#     print 'accuracy: ', np.mean(scores[len(columns)][0])
#     print 'loss: ', np.mean(scores[len(columns)][1])

#     print 'kls JS probabilities'
#     print 'accuracy: ', np.mean(scores[len(columns)+1][0])
#     print 'loss: ', np.mean(scores[len(columns)+1][1])

In [44]:
# from sklearn import linear_model
# from sklearn.model_selection import KFold
# from sklearn.metrics import log_loss,accuracy_score
# # cols is a list of column sets for logistic regression; 
# # probs are model-specific probabilities
# def validate_results(df,probs,lm_columns,n_splits=5):
#     kfold = KFold(n_splits=n_splits,shuffle=True)
#     scores = np.zeros([len(lm_columns)+len(probs),2,n_splits]);i=0
#     for train_ind,test_ind in kfold.split(df):
#         lm = linear_model.LogisticRegression(fit_intercept = True)
#         train_df,test_df = df.loc[train_ind],df.loc[test_ind]
        
#         for j,prob_col in enumerate(probs):
#             y_preds = test_df[prob_col]>.5
#             scores[j][0][i]=accuracy_score(test_df['winner'],y_preds)
#             scores[j][1][i]=log_loss(test_df['winner'],test_df[prob_col],labels=[0,1])
        
#         for k,cols in enumerate(lm_columns):
#             lm.fit(train_df[cols].values.reshape([len(train_df),len(cols)]),train_df['winner'])
#             y_preds = lm.predict(test_df[cols].values.reshape([len(test_df),len(cols)]))
#             y_probs = lm.predict_proba(test_df[cols].values.reshape([len(test_df),len(cols)]))
#             scores[len(probs)+k][0][i]=accuracy_score(test_df['winner'],y_preds)
#             scores[len(probs)+k][1][i]=log_loss(test_df['winner'],y_probs,labels=[0,1])
#         i+=1

#     for j,prob_col in enumerate(probs):
#         print prob_col
#         print 'accuracy: ', np.mean(scores[j][0])
#         print 'loss: ', np.mean(scores[j][1])
    
#     for i,cols in enumerate(lm_columns):
#         print 'lm columns: ',cols
#         print 'accuracy: ', np.mean(scores[len(probs)+i][0])
#         print 'loss: ', np.mean(scores[len(probs)+i][1])